In [41]:
import pandas as pd
import altair as alt
import warnings
import requests
import datetime
import matplotlib.pyplot as plt
import json
import requests
import numpy as np
alt.renderers.set_embed_options(theme='dark')
pd.set_option('display.max_colwidth', None)

In [87]:
class AstroDataProvider:
    
    def __init__(self, claim):
        daic_url = "https://terra-api.daic.capital/api/tx/GetRichlistByTokenContract?apiKey=vAp6ysmAXH470YcphYxv&contract_address={}"
        self.votes = '4940a215-6e93-4107-bf08-50574b3e431d'
        self.astro_holders_url =daic_url.format("terra1xj49zyqrwpv5k928jwfpfy2ha668nwdgkwlrg3")
        self.xastro_holders_url =daic_url.format("terra14lpnyzc9z4g3ugr4lhm8s4nle0tq8vcltkhzh7")
        self.claim = claim
        
    def get_from_url(self, url):
        json = requests.get(url).json()
        return json
        
    def load(self):
        self.votes_df = self.claim(self.votes)
        #
        json = self.get_from_url(self.astro_holders_url)['result']['holders']
        self.astro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.astro_holders_df.columns =  ['addr','amount']
        #
        json = self.get_from_url(self.xastro_holders_url)['result']['holders']
        self.xastro_holders_df = pd.DataFrame(json.values(),json.keys()).reset_index()
        self.xastro_holders_df.columns =  ['addr','amount']
        #
        
    def parse_proposal_recap(self):
        votes = self.votes_df.groupby(['proposal_id','vote']).sum().voting_power.reset_index()
        against = votes[votes.vote=='against']
        against.columns = ['proposal_id','against','voting_power_against']
        for_ = votes[votes.vote=='for']
        for_.columns = ['proposal_id','for','voting_power_for']
        votes = against.merge(for_, on='proposal_id')
        votes['delta'] = votes['voting_power_for'] - votes['voting_power_against'] 
        votes['result'] = votes.apply(lambda row: 'passed' if row.delta > 0 else 'failed', axis=1)
        votes['result'] = votes.apply(lambda row: 'passed' if row.delta > 0 else 'failed', axis=1)
        return votes
    
    def parse_top_active_voters(self):
        return dp.votes_df.groupby('voter').agg({'voting_power':'sum','tx_id':'count'})\
                            .sort_values(by=['tx_id','voting_power'], ascending=False)\
                            .head(20)
    def parse_dist_voting_power_per_proposal(self):
        return dp.votes_df[['proposal_id','voting_power']]\
            .pivot(columns='proposal_id',values='voting_power')
        
    def parse(self):
        self.astro_holders_df.amount = self.astro_holders_df.amount/1000000
        self.proposal_recap = self.parse_proposal_recap()
        self.top_active_voters = self.parse_top_active_voters()
        self.dist_voting_power_per_proposal = self.parse_dist_voting_power_per_proposal()
        
    def to_csv(self, path='../data'):
        self.votes_df.to_csv(f"{path}/votes_df.csv")
        self.astro_holders_df.to_csv(f"{path}/astro_holders_df.csv")
        self.proposal_recap.to_csv(f"{path}/proposal_recap.csv")
        self.top_active_voters.to_csv(f"{path}/top_active_voters.csv")
        self.dist_voting_power_per_proposal.to_csv(f"{path}/dist_voting_power_per_proposal.csv")
        
    def read_csv(self):
        url = 'https://raw.githubusercontent.com/IncioMan/nebula_lbp/master/data/{}.csv'
        self.ust_traded_prices_df =  pd.read_csv(url.format('ust_traded_prices_df'), index_col=0)


In [88]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [89]:
dp = AstroDataProvider(claim)
dp.load()
dp.parse()
dp.to_csv()

In [395]:
class NebulaChartProvider:
    
    def ust_traded_prices_chart(self, ust_traded_prices):
        chart = alt.Chart(ust_traded_prices).mark_point().encode(
        x=alt.X('Price:Q', sort=alt.EncodingSortField(order='ascending')),
        y="Amount UST (M):Q",
        color=alt.Color('Action:N', scale=alt.Scale(domain=['Sold NEB','Bought NEB'],
                                                      range=['#F24A72','#21bcd7'])),
        tooltip=['Action','Amount UST (M):N','Price:Q']
        ).configure_mark(
            color='#21bcd7'
        ).properties(width=700).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def first_price_chart(self,df):
        cols = ['Number of Users','Price']
        chart = alt.Chart(df).mark_line(point=True).encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":Q",axis=alt.Axis(tickCount=20, labelAngle=0, tickBand = 'center')),
            tooltip=[cols[0],cols[1]]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def first_time_chart(self,df):
        cols = ['Number of Users','Time'] 
        chart = alt.Chart(df).mark_bar().encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":T"),
            tooltip=[alt.Tooltip(cols[1]+':T', format='%Y-%m-%d %H:%M'), alt.Tooltip(cols[0]+":Q")]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def n_prices_per_users_df_chart(self,df):
        cols = ['Number of Users','Number of Different Prices']
        chart = alt.Chart(df).mark_bar().encode(
            y=alt.Y(cols[0]+":Q"),
            x=alt.X(cols[1]+":N",axis=alt.Axis(tickCount=10, labelAngle=30, tickBand = 'center')),
            tooltip=[cols[1], cols[0]]
        ).configure_mark(
            color='#21bcd7'
        ).properties(height=300).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart
    
    def user_distr_pie(self, df, cols):
        chart = alt.Chart(df).mark_arc(innerRadius=60).encode(
            theta=alt.Theta(field=cols[0], type="quantitative"),
            color=alt.Color(field=cols[1], type="nominal",
                    #sort=['MARS & UST','MARS','UST'],
                    scale=alt.Scale(domain=df[cols[1]].unique(), range=['#F24A72','#21bcd7']),
                    legend=alt.Legend(
                    orient='none',
                    padding=10,
                    legendY=-10,
                    direction='vertical')),
            tooltip=[cols[1]+':N',cols[0]+':N']
        ).configure_view(strokeOpacity=0)
        return chart
    
    def sender_airdrop_op_charts(self, df, cols):
        df.columns = cols
        chart = alt.Chart(df).mark_arc(innerRadius=60).encode(
                    theta=alt.Theta(field=cols[1], type="quantitative"),
                    color=alt.Color(field=cols[0], type="nominal",
                            #sort=['MARS & UST','MARS','UST'],
                            scale=alt.Scale(domain=df[cols[0]].unique(), range=['#ffffff','#21bcd7','#F24A72']),
                            legend=alt.Legend(
                            orient='none',
                            padding=10,
                            legendY=-10,
                            direction='vertical')),
                    tooltip=[cols[1]+':N',cols[0]+':N']
                ).configure_view(strokeOpacity=0)
        return chart
    
    def price_chart(self,hourly_stats_df):
        #272231 background
        df=hourly_stats_df[['avg_belief_price','time']]
        df.columns=['Price','Hour']
        n_data = 20
        if df.Hour.nunique() < n_data:
            extra_data = []
            for i in range(n_data-df.Hour.nunique()):
                extra_data.append([None,(pd.to_datetime(df.Hour.max())+datetime.timedelta(hours=i)).strftime("%Y-%m-%d %H:%M")])
            df2 = df.append(pd.DataFrame(extra_data, columns=df.columns))
        else:
            df2 = df
        chart = alt.Chart(df2).mark_line(point=True).encode(
            x=alt.X('Hour:T', sort=alt.EncodingSortField(order='ascending')),
            y="Price:Q",
            tooltip=['Hour:T',"Price:Q"]
        ).configure_mark(
            color='#21bcd7'
        ).properties(width=700).configure_axisX(
            labelAngle=0
        ).configure_view(strokeOpacity=0).configure_axis(grid=False)
        return chart